# Final Part

In the last part, we use the final tuned model to finally see the Kaggle score.

# Pre-processing the data again 

We will do the prepprocessing of the data as done on the previous part. However, instead of dividing the train data, we will use the test data provided by Kaggle this time.

In [1]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

#importing the  necessary modules
import pandas
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import zipfile
from sklearn import cross_validation, metrics


zip_ref = zipfile.ZipFile("train.csv.zip", 'r')
zip_ref.extractall()
zip_ref.close()

#loading train data to be used to create submission file to submission file to be used for kaggle submission
zip_ref2 = zipfile.ZipFile("test.csv.zip", 'r')
zip_ref2.extractall()
zip_ref2.close()


train_data = pandas.read_csv("train.csv")
test_data = pandas.read_csv("test.csv")
ids = test_data['id']
test_data['loss'] = np.nan


joined = pandas.concat([train_data, test_data],ignore_index = True)
del train_data,test_data

cat_feature = [n for n in joined.columns if n.startswith('cat')]

for column in cat_feature:
        joined[column] = pandas.factorize(joined[column].values, sort=True)[0]
        
del cat_feature



train_data = joined[joined['loss'].notnull()]
test_data = joined[joined['loss'].isnull()]
del joined

shift = 200
train_data["loss"] = np.log(train_data["loss"]+shift)
target = train_data['loss']
train_data.drop(['id','loss'],1,inplace=True)
test_data.drop(['id','loss'],1,inplace=True)


/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

## Training the model 

In [3]:
RANDOM_STATE = 2016
params = {
        'min_child_weight': 8,
        'eta': 0.01,
        'colsample_bytree': 0.45,
        'max_depth': 6,
        'subsample': 1.0,
        'alpha': 1,
        'gamma': 0,
        'silent': 1,
        'verbose_eval': True,
        'seed': RANDOM_STATE,'verbose_eval': 2}

xgtrain = xgb.DMatrix(train_data, label=target)
xgtest = xgb.DMatrix(test_data)

model = xgb.train(params, xgtrain, 3000, feval=evalerror)

prediction = np.exp(model.predict(xgtest)) - shift

Now, making a submission file for Kaggle servers.

In [5]:
submission = pandas.DataFrame()
submission['loss'] = prediction
submission['id'] = ids
submission.to_csv('sub_v.csv', index=False)

After submitting the Kaggle file, the score recieved was 1122.14212

## The End